# Projet Tribel #

### Explorer et mettre en forme le fichier CSV ###

Nous avons trouvé sur le site du gouvernement un [dataset](https://www.data.gouv.fr/fr/datasets/nomenclature-du-tri-selectif-prs/) sur lequel figure des informations à propos de plus de 110.000 produits mise en vente dans les supermarchés français.

Nous allons utiliser ce dataset afin d'obtenir plus d'informations sur le type d'emballage de ces produits et savoir si ils doivent être mis au tri ou à la poubelle. 

Le fichier original .csv fait plus de 3 Go, ce qui est trop lourd à processer pour excel sur nos ordinateurs de bureau. Nous allons donc utiliser un échantillon de cette base de donnée afin d'avoir un fichier plus léger et plus maniable. Nous avons décidé de garder 1000 produits, dans un fichier .xlsx (*base1000poubelle*). 

Un bon nombre de ces produits n'ont pas d'informations renseignées quant à la composition de leur packaging. Nous avons, lors de l'exctraction des données, gardé uniquement les 1000 premiers produits ayant des informations relatives à la composition du packaging. 

Ci-dessous nous ouvrons le fichier .csv et nous séparons la première ligne qui comporte les titres des colonnes du reste des données.




In [1]:
import csv
fh = open('base1000poubelle.csv',errors='ignore')
reader = csv.reader(fh, delimiter = ';', quotechar = '"')
dataset = list(reader)
dataset_header = dataset[:1]
dataset = dataset[1:]
print(len(dataset))
print(len(dataset[0]))

1000
178


In [2]:
print(dataset[2][10])

carton,boÃ®te,surgelÃ©


Nous nous rendons compte que le fichier comporte 178 colonnes, on se rend compte que la grande majorité de ses colonnes sont inutiles pour notre projet (ex : informations sur la nutrition...).


In [3]:
print(dataset_header[0])

['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutriscore_score', 'nutriscore_grade', '

Nous allons donc uniquement garder les éléments intéressants pour notre projet, soit les colonnes : 

1. code (colonne index : 0)
2. product_name (colonne index : 7)
3. packaging (colonne index : 10)
4. packaging_tags (colonne index : 11)
5. brands (colonne index : 12)
6. brands_tags (colonne index : 13)

On remarque que les colonnes avec les mentions tags correspondent aux mêmes infos que celles sans les tags, sans les erreurs

In [4]:
print(dataset[2][10])
print(dataset[2][11])

carton,boÃ®te,surgelÃ©
carton,boite,surgele


Nous allons donc garder uniquement les colonnes avec la mention tags :

1. code (colonne index : 0)
2. product_name (colonne index : 7)
3. packaging_tags (colonne index : 11)
4. brands_tags (colonne index : 13)

In [5]:
import numpy as np

dataset = np.array(dataset)
poubelle = dataset[:,[0,7,11,13]]

print(poubelle[0:5])


[['20114' 'Naturablue original' 'bocal,plastique' 'natura4ever']
 ['10127735' 'Sirops pour ganache macarons' 'carton,bouteille,verre'
  'alice-delice']
 ['69160028' 'Big Choco Framboise' 'carton,boite,surgele' 'bofrost']
 ['30' 'Cakes Raisins' 'boite,carton,plastique,sachet' 'bijou']
 ['31' 'Cakes aux Fruits' 'boite,carton,sachet,plastique' 'bijou']]


Utilisons à présent Pandas qui nous permettra de manipuler nos informations plus simplement.

In [6]:
import pandas as pd
poubellepd = pd.DataFrame(poubelle, columns = ['code','product_name','packaging','brands' ])
print(poubellepd.head(10))  
poubellepd.info()                      

       code                           product_name  \
0     20114                    Naturablue original   
1  10127735           Sirops pour ganache macarons   
2  69160028                    Big Choco Framboise   
3        30                          Cakes Raisins   
4        31                       Cakes aux Fruits   
5     39365      Pure Whey Protein Berries & Cream   
6     39681  Pure Whey Proteinâ„¢ Chocolate Peanut   
7        50                 Financiers aux Amandes   
8        51                        Fondants Citron   
9       104                      30 Panach' Fruits   

                                           packaging        brands  
0                                    bocal,plastique   natura4ever  
1                             carton,bouteille,verre  alice-delice  
2                               carton,boite,surgele       bofrost  
3                      boite,carton,plastique,sachet         bijou  
4                      boite,carton,sachet,plastique        

We decide to get rid of all the column where the packaging is not relevant to our goal. We erase the column that do not contains one of those words : "plastique|carton|verre|boite|glass|metal|barquette"

In [7]:
print(poubellepd.head())
poubellepd = poubellepd[poubellepd["packaging"].str.contains("plastique|carton|verre|boite|glass|metal|barquette", na =False, regex = True)]
poubellepd.head()

       code                  product_name                      packaging  \
0     20114           Naturablue original                bocal,plastique   
1  10127735  Sirops pour ganache macarons         carton,bouteille,verre   
2  69160028           Big Choco Framboise           carton,boite,surgele   
3        30                 Cakes Raisins  boite,carton,plastique,sachet   
4        31              Cakes aux Fruits  boite,carton,sachet,plastique   

         brands  
0   natura4ever  
1  alice-delice  
2       bofrost  
3         bijou  
4         bijou  


,code,product_name,packaging,brands
0,20114,Naturablue original,"bocal,plastique",natura4ever
1,10127735,Sirops pour ganache macarons,"carton,bouteille,verre",alice-delice
2,69160028,Big Choco Framboise,"carton,boite,surgele",bofrost
3,30,Cakes Raisins,"boite,carton,plastique,sachet",bijou
4,31,Cakes aux Fruits,"boite,carton,sachet,plastique",bijou


In [8]:
poubellepd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908 entries, 0 to 999
Data columns (total 4 columns):
code            908 non-null object
product_name    908 non-null object
packaging       908 non-null object
brands          908 non-null object
dtypes: object(4)
memory usage: 35.5+ KB


Il nous reste 908 entrées dans notre Dataset. Nous allons particulièrement nous intéresser à la colonne packaging_tags, qui nous indique si l'emballage contient du plastique, du carton, etc...

Isolons chacun des termes présents dans cette colonne : 

In [9]:
poubellepd["packaging"]= poubellepd["packaging"].str.split(",", n=6 , expand = False) 
poubellepd.head(30)

,code,product_name,packaging,brands
0,20114,Naturablue original,"[bocal, plastique]",natura4ever
1,10127735,Sirops pour ganache macarons,"[carton, bouteille, verre]",alice-delice
2,69160028,Big Choco Framboise,"[carton, boite, surgele]",bofrost
3,30,Cakes Raisins,"[boite, carton, plastique, sachet]",bijou
4,31,Cakes aux Fruits,"[boite, carton, sachet, plastique]",bijou
5,39365,Pure Whey Protein Berries & Cream,"[plastique, doypack, sachet, pochon, sachet-re...",bulk-powders
6,39681,Pure Whey Proteinâ„¢ Chocolate Peanut,"[sachet, plastique, doypack, sachet-refermable]",bulk-powders
7,50,Financiers aux Amandes,"[boite-carton, 30-emballages-individuels]",bijou
8,51,Fondants Citron,"[boite-carton, sachet-plastique]",bijou
9,104,30 Panach' Fruits,"[boite, carton, plastique]",bijou


Nous allons de plus apporter quelques modifications au nom des produits, dont certains comportent des caractères spéciaux.

In [10]:
poubellepd["product_name"] = poubellepd["product_name"].astype(str)
poubellepd["product_name"] = poubellepd["product_name"].str.replace("Ã©","é").str.replace("Ã®","î").str.replace("Ã§","ç")
poubellepd.head(50)

,code,product_name,packaging,brands
0,20114,Naturablue original,"[bocal, plastique]",natura4ever
1,10127735,Sirops pour ganache macarons,"[carton, bouteille, verre]",alice-delice
2,69160028,Big Choco Framboise,"[carton, boite, surgele]",bofrost
3,30,Cakes Raisins,"[boite, carton, plastique, sachet]",bijou
4,31,Cakes aux Fruits,"[boite, carton, sachet, plastique]",bijou
5,39365,Pure Whey Protein Berries & Cream,"[plastique, doypack, sachet, pochon, sachet-re...",bulk-powders
6,39681,Pure Whey Proteinâ„¢ Chocolate Peanut,"[sachet, plastique, doypack, sachet-refermable]",bulk-powders
7,50,Financiers aux Amandes,"[boite-carton, 30-emballages-individuels]",bijou
8,51,Fondants Citron,"[boite-carton, sachet-plastique]",bijou
9,104,30 Panach' Fruits,"[boite, carton, plastique]",bijou


Nous pouvons remarquer que dans la colonne "Code" il y a des longueurs de codes qui ne correspondent pas à ce qui est attendu par notre lecteur de code barres (nous cherchons des codes barres qui soit reconnaissables : EAN ou UPC, c'est à dire avec 8-12-13-15 chiffres). Regardons de plus près cette colonne.

Nous allons donc supprimer les lignes avec des Codes qui ne sont pas entre 8 et 15 chiffres

In [11]:
poubellepd2 = poubellepd[poubellepd["code"].str.len() < 16]
poubellepd2 = poubellepd[poubellepd["code"].str.len() > 8]
poubellepd2["code"].describe()
poubellepd2
    

,code,product_name,packaging,brands
29,11200009717,Green Pepper Sauce,"[bouteille, verre, carton, boite]","tabasco,mc-ilhenny-co"
30,11210601369,Green Pepper Sauce,"[flacon, bouteille, verre]","tabasco,mc-ilhenny-co"
31,11210607040,Tabasco Chipotle,"[boite, carton, bouteille, verre]",mccilhenny
34,11747615358,æ–°äºžå§œç³– è–„è·å‘³,"[fr-boite-carton, fr-film-plastique]",sina-peppermint
35,12000012785,Diet Green Tea Citrus,"[bouteille, plastique]","lipton,unilever"
38,14100042303,COOKIES AU CHOCOLAT NOIR,"[plastique, boite, papier-d-emballage, plastiq...","pepperidge-farm,milano"
39,14100085478,Goldfish Baked Snack Crackers Cheddar,"[sachet, plastique, papier, bolsa-de-plastico]",pepperidge-farm
42,16000123991,,"[carton, boite, plastique]",general-mills
43,16000141568,Lucky Charms,"[cup, gobelet, plastique, ldpe]",general-mills
44,16000264601,Crunchy Granola Bars Oats 'n Honey,"[sachet, plastique, boite, carton]","nature-valley,general-mills"


Nous pouvons voir que 159 lignes ont pour code 2000000000000 , la base de données ne doit pas être correcte sur ce point car 159 produits ne peuvent pas avoir le même code barre. Nous allons donc supprimer ces lignes pour ne garder que les produits avec un code barre qui ait du sens pour notre projet.

Pour cela nous avons besoin de transformer les données de la colonne "Code" de string à integer.

In [12]:
poubellepd2["code"] = poubellepd2["code"].astype("int64")


C:\Users\Pikto\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
poubellepd2 = poubellepd2[poubellepd2["code"] != 2000000000000]


Cependant nous nous rendons compte avec "value_counts" qu'il existe d'autres valeurs en doublon dans notre colonne "code". Nous n'allons pas les supprimer une par une mais nous utiliserons la fonction "drop_duplicates".

In [23]:
poubellepd2["code"].value_counts().head(10)

613009000000     13
1200100000000     4
845163000000      4
654479000000      4
812475000000      3
687456000000      2
896899000000      2
87521300008       2
655762000000      2
652655000000      2
Name: code, dtype: int64

In [24]:
poubellepd2.drop_duplicates(subset ="code", keep = False, inplace = True) 

In [25]:
poubellepd2["code"].value_counts().head(10)

70177029623      1
1400000000000    1
38000766114      1
96619897001      1
39047013551      1
37617390248      1
217904000000     1
775462000000     1
61400000182      1
71000001724      1
Name: code, dtype: int64

Finalement nous avons juste besoin de remettre notre index à 0 afin de finaliser notre base de données.

In [28]:
poubellepd2 = poubellepd2.reset_index(drop=True)
poubellepd2

,code,product_name,packaging,brands
0,11200009717,Green Pepper Sauce,"[bouteille, verre, carton, boite]","tabasco,mc-ilhenny-co"
1,11210601369,Green Pepper Sauce,"[flacon, bouteille, verre]","tabasco,mc-ilhenny-co"
2,11210607040,Tabasco Chipotle,"[boite, carton, bouteille, verre]",mccilhenny
3,11747615358,æ–°äºžå§œç³– è–„è·å‘³,"[fr-boite-carton, fr-film-plastique]",sina-peppermint
4,12000012785,Diet Green Tea Citrus,"[bouteille, plastique]","lipton,unilever"
5,14100042303,COOKIES AU CHOCOLAT NOIR,"[plastique, boite, papier-d-emballage, plastiq...","pepperidge-farm,milano"
6,14100085478,Goldfish Baked Snack Crackers Cheddar,"[sachet, plastique, papier, bolsa-de-plastico]",pepperidge-farm
7,16000123991,,"[carton, boite, plastique]",general-mills
8,16000141568,Lucky Charms,"[cup, gobelet, plastique, ldpe]",general-mills
9,16000264601,Crunchy Granola Bars Oats 'n Honey,"[sachet, plastique, boite, carton]","nature-valley,general-mills"
